# <center>Команда СТО11</center>

## Обработка данных
Работу с данными начнем с получения таблицы пост-тэг-количество постов, помеченных этим тэгом

In [11]:
import pandas as pd
posts = pd.read_csv('posts.csv',  encoding = "ISO-8859-1")
tags_forR=posts[['Id','Tags']]
tags_cols=pd.Series([[i.strip('>') for i in d.split('<') if len(i) > 0] for d in tags_forR.Tags if type(d) is str])
tags_forR['Tag']=tags_cols
tags_forR = tags_forR.set_index(['Id'])['Tag'].apply(pd.Series).stack()
tags_forR = tags_forR.reset_index()
tags_forR.columns=['Id','TagN','Tag']
tags_forR=tags_forR[['Id','Tag']]
tags_forR.head()
tags_forR.to_csv("Tags_stack.csv",index=False)

C:\Users\User\Anaconda2\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Оставим теги, представленные более, чем в 700 постах

In [12]:
df=pd.read_csv("Tags_stack.csv")
diccounts=df.Tag.value_counts().to_dict()
df["Tagcounts"] = df["Tag"].map(diccounts)
df = df[df['Tagcounts']>700]
df['Tag'].value_counts().head(10)

javascript    29072
java          26008
c#            22620
php           22009
android       20541
jquery        17613
python        16193
html          13707
ios           10455
c++           10295
Name: Tag, dtype: int64

Из тела поста требуется удалить html-тэги, элементы форматирования, знаков препинания. Важно иметь ввиду, что некоторые символы, не являющиеся буквами или цифрами, также могут входить в важные с точки зрения модели токены (c++, ASP.NET, ...)

In [13]:
from bs4 import BeautifulSoup as bs
import re

def clean_code(text1):
    
    text = bs(text1, 'html.parser')
    
    def cleanhtml(raw_html):
        cleanr = re.compile('<.*?>')
        cleanr2 = re.compile(r'\r')
        cleanr3 = re.compile(r' : ')
        cleanr4 = re.compile(r'  ')
        cleanr5 = re.compile(r'\n')

        cleantext = re.sub(cleanr, '', raw_html)
        cleantext = re.sub(cleanr2, '\n', cleantext)
        cleantext = re.sub(cleanr3, ': ', cleantext)
        cleantext = re.sub(cleanr5, ' ', cleantext)
        
        return cleantext.strip('\n')
    
    q = text1
    for code in text.find_all('code'):
        q = q.replace(str(code), '')
    
    return cleanhtml(q)

def clean_punct(t):
    pattern = re.compile(r'[#+-.a-zA-Z0-9]+')
    t = re.findall(pattern, t)
    res = ' '.join(t).lower()
    res = re.sub(r'\.$', '', res)
    res = re.sub(r'\s*\.\s+', ' ', res)
    res = re.sub(r',', '', res)
    return ''.join(res)
posts['new_body'] = posts['Body'].apply(clean_code)
posts['new_body_1'] = posts['new_body'].apply(clean_punct)

Заголовки (где они есть) были очищены от знаков препинания

In [14]:
posts['new_title'] = ''
posts.loc[~posts['Title'].isnull(), 'new_title'] = posts.loc[~posts['Title'].isnull(), 'Title'].apply(clean_punct)

Итоговое представление поста - объединение title и body

In [15]:
posts['all_text'] = posts['new_title'] + ' ' + posts['new_body_1']

К тексту применялся стемминг. Это позволило сократить размерность пространства признаков и сделать их более статистически достоверными. Датасет здесь был разбит на две части: вопросы и ответы.

In [16]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
questions = posts[posts['PostTypeId']==1]
print(questions.shape)
ps = PorterStemmer()
def stem(t):
    l = t.split()
    res = [ps.stem(w) for w in l]
    return ' '.join(res)
questions['all_text_stemmed'] = questions['all_text'].apply(stem)
answers = posts[posts['PostTypeId']==2]
answers['all_texty_stemmed'] = answers['all_text'].apply(stem)

(289122, 25)


C:\Users\User\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


При построении tfidf мы использовали слова и биграммы. Размерность признакового пространства получилась порядка 10е6, что нормально для пободных задач.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
vzer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
X = vzer.fit_transform(questions['all_text_stemmed'])
X_ans = vzer.transform(answers['all_texty_stemmed'])
X.shape

(289122, 3665569)

Поскольку выборка не сбалансирована, не будем смотреть на accuracy. Точность на тэге javascript составила 0.74, полнота 0.46. У нашего алгоритма есть некоторые трудности с полнотой выделения объектов положительного класса. Это связано скорее всего с тем, что в проблемных постах отсутствуют наиболее сильные слова-предикторы определенных тэгов.

Нужно сказать, что глобальная цель - это все-таки не предсказание тэгов, а получение векторных представлений постов и оценка их близости.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
y = (questions['Id'].isin(df.loc[df['Tag']=='javascript', 'Id']).astype(int))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
est = LinearSVC()
est.fit(X_train, y_train)
print(precision_score(y_test, est.predict(X_test)))
print(recall_score(y_test, est.predict(X_test)))

0.74435066703
0.470244238046


Здесь мы получаем векторные представления постов. Элементами векторов будут решающие функции, оценивающие соответствие тэга посту.

In [19]:
import gc
import numpy as np
gc.collect()
ans_embeddings = pd.DataFrame(columns=np.unique(df['Tag'].values), index=answers.index)
questions_embeddings = pd.DataFrame(columns=np.unique(df['Tag'].values), index=questions.index)
for tag in np.unique(df['Tag'].values):
    y = questions['Id'].isin(df.loc[df['Tag']==tag, 'Id']).astype(int)
    est.fit(X, y)
    pred_ans = est.decision_function(X_ans)
    pred_question = est.decision_function(X)
    ans_embeddings.loc[:, tag] = pred_ans
    questions_embeddings.loc[:, tag] = pred_question
    print(tag)

.htaccess
.net
actionscript-3
ajax
algorithm
amazon-web-services
android
android-layout
android-studio
angular
angularjs
apache
api
arrays
asp.net
asp.net-mvc
asp.net-mvc-3
asp.net-mvc-4
azure
bash
batch-file
c
c#
c++
c++11
class
cocoa
codeigniter
cordova
css
css3
csv
database
date
datetime
debugging
delphi
dictionary
django
eclipse
email
entity-framework
excel
excel-vba
facebook
file
flash
forms
function
git
google-app-engine
google-chrome
google-maps
haskell
hibernate
html
html5
http
image
ios
ipad
iphone
java
javascript
jquery
jquery-ui
json
jsp
laravel
linq
linux
list
listview
loops
magento
matlab
maven
mongodb
multithreading
mysql
node.js
numpy
objective-c
oop
opencv
oracle
osx
pandas
parsing
performance
perl
php
pointers
postgresql
powershell
python
python-2.7
python-3.x
qt
r
reactjs
regex
rest
ruby
ruby-on-rails
ruby-on-rails-3
scala
security
selenium
session
shell
sockets
sorting
spring
spring-mvc
sql
sql-server
sql-server-2008
sqlite
string
swift
swing
symfony
templates
tsql
t

Векторные представления вопросов выглядят так:

In [20]:
questions_embeddings

,.htaccess,.net,actionscript-3,ajax,algorithm,amazon-web-services,android,android-layout,android-studio,angular,...,visual-studio-2010,wcf,web-services,windows,winforms,wordpress,wpf,xaml,xcode,xml
0,-1.290281,-1.222134,-0.929959,-1.145873,-1.290138,-1.030694,-1.153978,-1.248949,-1.232214,-1.097537,...,-1.358331,-1.214681,-1.205395,-1.103991,-1.261423,-1.163033,-1.388066,-1.211152,-1.358413,0.589760
1,-1.376430,-1.201913,-1.354798,-1.401800,-0.883318,-1.132355,0.317022,-1.229178,-1.222433,-1.285350,...,-1.324759,-1.240663,-1.245844,-0.950242,-1.447221,-1.429809,-1.345666,-1.278107,-1.521973,-1.195937
2,-1.439299,-1.048577,-1.266459,-0.782230,-1.252516,-1.365816,-1.666446,-1.399057,-1.207440,-1.237060,...,-1.344423,-1.187650,-1.241144,-1.380855,-1.535110,-1.050331,-1.613347,-1.488610,-1.378442,-1.257392
3,-1.136201,0.319562,-1.214566,-1.441422,-1.379534,-1.232878,-1.322198,-1.218289,-1.270415,-1.199239,...,-1.091979,-0.974427,-1.016485,-0.987424,-1.202781,-1.142292,-1.255993,-1.239925,-1.106856,-1.158946
4,-1.425440,-0.848000,-1.232082,-1.172171,-1.314835,-1.330210,-1.314836,-1.315163,-1.145053,-0.996708,...,-1.083968,0.934593,-0.996392,-1.209505,-1.380567,-1.320791,-1.339620,-1.308388,-1.278491,-1.324928
5,-1.262053,-1.280707,-1.138192,-1.110776,-1.089963,-0.999209,0.230777,-1.291213,-1.282917,-1.093867,...,-1.333251,-1.259273,-1.169906,-1.263969,-1.170960,-1.166961,-1.146450,-1.141348,-0.987976,-1.404272
6,-1.330021,-1.173399,-1.081437,-1.348065,-1.297991,-1.344980,-1.315445,-1.410690,-1.109069,-1.052844,...,-1.227999,-1.286168,-1.284553,-1.044465,-1.281852,-1.411296,-1.056269,-1.302086,-1.269497,-1.073986
7,-1.220595,-1.272841,-1.268230,-1.035732,-1.335362,-1.377229,-1.469450,-1.169066,-1.182848,-1.324825,...,-1.162085,-1.261540,-1.151111,-1.294920,-1.131051,-1.320072,-1.288070,-1.206999,-1.354362,-1.136309
8,-1.426742,-1.420815,-1.204816,-1.172028,-1.240557,-1.208548,-1.475928,-1.411503,-1.213869,-1.372641,...,-1.304918,-1.283329,-1.234767,-1.170552,-1.371087,-1.472202,-1.221708,-1.220702,-1.263714,-0.858078
9,-1.312845,-1.218661,-1.084331,-1.143452,-1.512212,-1.246757,0.796900,-1.245568,-1.231123,-0.991065,...,-1.230205,-1.171883,-1.329002,-1.313808,-1.221245,-1.328499,-1.606937,-1.209347,-1.302267,-1.410862


Близость векторов будем оценивать, используя косинусную меру. Построим косинусные расстояния для вопроса 6 и всех ответов. Выведем номера ответов в порядке убывания косинусного растояния. По этим номерам мы можем найти соответствующие ответы в answers и по описанию убедиться, что близость оценивается адекватно.

In [21]:
from tqdm import tqdm
from numpy.linalg import norm
dist_series = pd.Series(index=answers.index)
q = questions_embeddings.loc[6, :].values
q_norm = norm(q)
for i in tqdm(range(len(answers))):
    ans_i = ans_embeddings.loc[answers.index[i]].values
    dist_series.loc[answers.index[i]] = (ans_i * q).sum() / norm(ans_i) / q_norm

100%|████████████████████████████████| 451429/451429 [01:49<00:00, 4112.55it/s]


In [22]:
dist_series.sort_values(ascending=False)

440297    0.996352
381678    0.995111
615616    0.995024
410987    0.994923
725100    0.994828
693352    0.994738
393944    0.994591
468068    0.994475
520326    0.994458
401327    0.994319
544547    0.994264
624873    0.994182
324668    0.994172
727967    0.994152
727060    0.994146
493685    0.994139
519691    0.994131
301393    0.994122
454323    0.994122
686630    0.994108
690159    0.994087
714931    0.994041
665605    0.994033
541578    0.994021
451515    0.993952
573133    0.993930
552720    0.993928
731341    0.993895
329811    0.993870
694599    0.993845
            ...   
497343    0.741293
341307    0.741293
507555    0.741293
316878    0.741293
719082    0.741293
323128    0.726153
527250    0.726153
331429    0.726153
672946    0.726153
606068    0.726129
711570    0.716955
303127    0.716955
364380    0.716955
349511    0.709283
331655    0.709283
317949    0.709283
706535    0.709283
545852    0.709283
405274    0.709283
329047    0.709283
423158    0.709283
695177    0.

Таким же образом, мы можем оценить близость вопроса другим вопросам.

In [23]:
dist_series1 = pd.Series(index=questions.index)
q = questions_embeddings.loc[288861, :].values
q_norm = norm(q)
for i in tqdm(range(len(questions))):
    ans_i = questions_embeddings.loc[questions.index[i]].values
    dist_series1.loc[questions.index[i]] = (ans_i * q).sum() / norm(ans_i) / q_norm

100%|████████████████████████████████| 289122/289122 [01:00<00:00, 4765.10it/s]


In [24]:
dist_series1.sort_values(ascending=False)

288861    1.000000
225683    0.995510
233017    0.995251
167812    0.994971
40686     0.994617
254957    0.994604
232196    0.994576
231831    0.994569
38669     0.994560
20664     0.994546
135750    0.994544
164481    0.994504
86985     0.994483
85944     0.994480
77767     0.994446
75779     0.994446
50808     0.994418
42943     0.994360
69508     0.994338
94391     0.994334
172772    0.994325
18        0.994314
95414     0.994309
225       0.994294
240411    0.994279
58285     0.994248
20406     0.994233
237559    0.994222
252093    0.994213
50255     0.994212
            ...   
521       0.875805
257232    0.875487
97677     0.875359
99898     0.875229
198521    0.874834
247374    0.874785
166820    0.873715
178746    0.872755
200556    0.872662
63297     0.872606
269869    0.872373
169910    0.871176
143829    0.869979
286936    0.868966
222547    0.867278
281627    0.866506
275466    0.866230
157099    0.865834
40343     0.865480
161981    0.863799
251610    0.862798
58401     0.